This notebook is for coursera Capstone

In [3]:
import pandas as pd
import numpy as np
print ("hello Capstone Project Course")

hello Capstone Project


In [5]:
import git
git remote add origin https://github.com/GeorgePapajim/Coursera_Capstone.git
git push -u origin master

SyntaxError: invalid syntax (<ipython-input-5-31f96380e31f>, line 2)